In [1]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import time

In [2]:
# 為資源的 Azure 端點和訂用帳戶金鑰建立變數
ENDPOINT = "https://0217-1.cognitiveservices.azure.com/"
training_key = "f78fb297caf64e1db1b532723952db92"
#training_images = "<D:\styler\train\image>"
prediction_key = "bc05678fd32e4271b396b4938b9315af"
prediction_resource_id = "8f4b15f6-5895-4afe-93e2-c87116a788a7"

In [3]:
#驗證用戶端
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [4]:
#建立新的視覺專案
publish_iteration_name = "classifyModel"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

#Create a new project
print ("Creating project...")
project = trainer.create_project("test", "0732100f-1a38-4e49-a514-c9b44c697ab5")

Creating project...


CustomVisionErrorException: Operation returned an invalid status code 'PermissionDenied'

In [5]:
#將標記新增至專案
black_tag = trainer.create_tag(project.id, "black")
white_tag = trainer.create_tag(project.id, "white")

NameError: name 'project' is not defined

In [6]:
#上傳和標記影像
base_image_location = "D:/styler/"

print("Adding images...")

image_list = []

for image_num in range(1, 11):
    file_name = "black_{}.jpeg".format(image_num)
    with open(base_image_location + "black/" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[black_tag.id]))

for image_num in range(1, 11):
    file_name = "white_{}.jpeg".format(image_num)
    with open(base_image_location + "white/" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[white_tag.id]))

upload_result = trainer.create_images_from_files("4b5746c2-c63c-4367-be51-335d6378440d", ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Adding images...


FileNotFoundError: [Errno 2] No such file or directory: 'D:/styler/black/black_1.jpeg'

In [7]:
#為專案定型 最少要放五張圖
print ("Training...")
iteration = trainer.train_project("4b5746c2-c63c-4367-be51-335d6378440d")
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration("4b5746c2-c63c-4367-be51-335d6378440d", " b58eda6d-9ccc-46bc-8f32-7d523d09898c")
    print ("Training status: " + iteration.status)
    time.sleep(1)

Training...


CustomVisionErrorException: Operation returned an invalid status code 'PermissionDenied'

In [8]:
# 發佈目前的反覆項目   這部分還沒成功
trainer.publish_iteration("4b5746c2-c63c-4367-be51-335d6378440d", "b58eda6d-9ccc-46bc-8f32-7d523d09898c", "TESTONE", "8f4b15f6-5895-4afe-93e2-c87116a788a7")
print ("Done!")

CustomVisionErrorException: Operation returned an invalid status code 'PermissionDenied'